# 01 - Find Files

**Search for the Files**
_______
Formatting the Directory-Pattern Dictionary
The function glob_multiple_file_paths expects a dictionary where each key-value pair corresponds to a root directory and a file pattern to search within that directory. The keys are the root directories where you want to start the search, and the values are the file patterns to match against.

Example Dictionary Format:

>dir_pattern_dict = {
>    '/path/to/first/root_dir': '*.nii',
>
>    '/path/to/second/root_dir': '*.nii.gz',
>
>    '/another/path': '*_label.nii'
>     Add more key-value pairs as needed
>}

Using Wildcards:

The file patterns can include wildcards to match multiple files:
- *: Matches zero or more characters
- **: Searches all directories recursively
- *.nii will match all files ending with .nii
- ?: Matches any single character
- file?.nii will match file1.nii, file2.nii, etc.
- [seq]: Matches any character in seq
- file[1-3].nii will match file1.nii, file2.nii, file3.nii
- [!seq]: Matches any character NOT in seq
- file[!1-3].nii will match any file that doesn't have 1, 2, or 3 in that position, like file4.nii, file5.nii, etc.

Feel free to combine these wildcards to create complex file patterns. For example, *_??.nii will match files like file_01.nii, file_02.nii, etc.

Where to Save to

In [ ]:
# Define the dictionary with root directories and file patterns
dir_pattern_dict = {
    '/Volumes/HowExp/datasets/05c_Howard_MetaAlzheimerReview_DBS-TMS_Coordinates/derivatives/roi/sub-*/ses-01/roi/': 'sub-*5.0mm_sphere_roi_composite.nii.gz'}

## Glob the files and check to see if acceptable

In [ ]:
save_files = False

In [ ]:
from calvin_utils.file_utils.file_path_collector import glob_multiple_file_paths
import os
# Validate Directory
# os.mkdir(os.path.dirname(csv_path))
# Call the function and save the returned DataFrame to a CSV file
path_df = glob_multiple_file_paths(dir_pattern_dict, save=save_files, save_path=None)

# Display the saved path and the DataFrame
display(path_df)

# Option 2 - Extract Subject IDs

In [ ]:
# Define the preceding and proceeding strings
preceding = 'sub-'
proceeding = '/'

# Extract the substring and add it to a new column 'subject'
path_df['subject'] = path_df['paths'].str.extract(f'{preceding}(.*?){proceeding}')

# Display the updated DataFrame
display(path_df)

In [ ]:
path_df['Subject'] = path_df['subject'].astype('str')

In [ ]:
path_df

# 03 Option A - Import Another CSV and Add the Paths to It
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [ ]:
spreadsheet_path = '/Volumes/HowExp/datasets/05c_Howard_MetaAlzheimerReview_DBS-TMS_Coordinates/metadata/master_list.xlsx'
sheet = 'Sheet1' #If using Excel, enter a string here

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=spreadsheet_path, output_dir=os.path.dirname(spreadsheet_path), sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

In [ ]:
merged_df = data_df.merge(path_df, on='Subject', how='outer', suffixes=('', '_pathdf'))
display(merged_df)

Rename Paths Column

In [ ]:
merged_df = merged_df.rename(columns={'paths': 'roi_path_old'})
display(merged_df)

Save the merged df

In [ ]:
merged_df.to_excel(spreadsheet_path, index=False)

# 03 Option B - Save Path DF To Its Own CSV

In [ ]:
# import os
# os.makedirs(out_dir, exist_ok=True)
# path_df.to_csv(os.path.join(out_dir, filename), index=False)

Hope this was helpful

--Calvin